In [7]:
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
from datetime import datetime
 
# Função para buscar e adicionar dados incrementais
def update_molybdenum_prices(csv_path):
    # Configurações do Chrome
    chrome_options = Options()
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-popup-blocking")
    chrome_options.add_argument("--disable-infobars")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-sandbox")
 
    driver = webdriver.Chrome(options=chrome_options)
 
    # URL do site da FGV
    page_url = "https://ycharts.com/indicators/molybdenum_spot_price"
    driver.get(page_url)
    print("Site abriu com sucesso")
 
    # Aguardar o carregamento completo da página
    WebDriverWait(driver, 50).until(
        EC.presence_of_element_located((By.XPATH, '//table[@class="table"]'))
    )
 
    # Extraindo as linhas da tabela de dados históricos
    rows = driver.find_elements(By.XPATH, '//table[@class="table"]/tbody/tr')
 
    # Criar uma lista para armazenar os novos dados
    new_data = []
 
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, "td")
        if len(cells) == 2:  # Garantir que a linha tenha duas células: data e valor
            date = cells[0].text.strip()
            value = cells[1].text.strip()
 
            # Filtrar valores inválidos
            if value.replace(".", "", 1).isdigit():  # Verifica se o valor é um número
                new_data.append({
                    "Date": date,
                    "Value": float(value.replace(",", ""))  # Remover vírgula para garantir que o valor seja numérico
                })
 
    # Criar DataFrame com os dados extraídos
    new_df = pd.DataFrame(new_data)
 
    # Converter as datas para o formato datetime para ordenação
    new_df['Date'] = pd.to_datetime(new_df['Date'], errors='coerce')
 
    # Ordenar os dados por data em ordem crescente
    new_df_sorted = new_df.sort_values(by="Date", ascending=True)
 
    # Limpar as últimas linhas que não são relevantes
    new_df_sorted = new_df_sorted.dropna(subset=['Date'])
 
    # Remover os últimos dados acumulados ou inválidos (se houver)
    new_df_sorted = new_df_sorted[new_df_sorted['Value'] != 44092.45]
 
    # Lê o arquivo CSV existente (se houver)
    try:
        existing_df = pd.read_csv(csv_path, sep=';')
        existing_df['Date'] = pd.to_datetime(existing_df['Date'], dayfirst=True, errors='coerce')
    except FileNotFoundError:
        # Se o arquivo não existir, começamos com um DataFrame vazio
        existing_df = pd.DataFrame(columns=["Date", "Value"])
 
    # Encontrar as últimas datas no arquivo CSV existente
    if not existing_df.empty:
        last_date_in_csv = existing_df['Date'].max()
    else:
        last_date_in_csv = datetime(1900, 1, 1)  # Se o CSV estiver vazio, começa com uma data antiga
 
    # Filtrar novos dados que não estão no CSV (dados mais recentes)
    new_entries = new_df_sorted[new_df_sorted['Date'] > last_date_in_csv]
 
    # Se houver novos dados, adicioná-los ao arquivo existente
    if not new_entries.empty:
        updated_df = pd.concat([existing_df, new_entries], ignore_index=True)
 
        # Ordenar novamente os dados (incluindo os novos) por data
        updated_df = updated_df.sort_values(by="Date", ascending=True)
 
        # Substituir os pontos por vírgulas para formatação no CSV final
        updated_df['Value'] = updated_df['Value'].apply(lambda x: str(x).replace('.', ','))
 
        # Salvar o CSV atualizado com separador ponto e vírgula
        updated_df.to_csv(csv_path, index=False, sep=';')
 
        print(f"Dados atualizados com sucesso! Novo arquivo salvo em {csv_path}")
    else:
        print("Não há novos dados para adicionar.")
 
    # Fechar o navegador
    driver.quit()
 
# Caminho do arquivo CSV a ser atualizado
csv_path = r"xxxx"
 
# Atualizar os dados
update_molybdenum_prices(csv_path)

Site abriu com sucesso
Não há novos dados para adicionar.
